# Part 2:  Extract from TMDB

- There is no financial information included in the IMDB data (e.g. budget or revenue). Analyze which movies are successful. The Movie Database (TMDB) as a great source of financial data
- Extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R) "Certification".


## Using tmdbsimple Package
2
- We will be using a Python package called tmdbsimple that will make it easier to extract the data we need without manually constructing the URLs for our API calls.

In [1]:
import pandas as pd
import numpy as np
import os,json

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [3]:
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'filtered_akas.csv.gz',
 'filtered_basics.csv.gz',
 'filtered_ratings.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [4]:
with open('/Users/scyjt/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [5]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
                
        # Sets file's current position at offset.
        file.seek(0)
        
        # convert back to json.
        json.dump(file_data, file)

In [6]:
# define custom dunction
def get_movie_with_rating(movie_id):

    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
        
    return info

In [7]:
new_basics = pd.read_csv("Data/filtered_basics.csv.gz", low_memory = False)

YEARS_TO_GET = [2000, 2001]

errors = [ ]

In [8]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

# If it does not exist: create it
if file_exists == False:
    
# save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
#Saving new year as the current df
df = new_basics.loc[new_basics['startYear']==YEAR].copy()

# saving movie ids to list
movie_ids = df['tconst'].copy()

In [10]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

In [11]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [12]:
#Get index and movie id from list
# INNER Loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
    try:
        # Retrieve the data for the movie id
        temp = get_movie_with_rating(movie_id)
            
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
            
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
            
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2001:   0%|          | 0/244 [00:00<?, ?it/s]

In [13]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")

- Total errors: 244
